# Quick and dirty code for plotting fov of the same mouse across all days

In [ ]:
import os
import tifffile as tiff
import numpy as np

In [ ]:
mouse = 'jm060'

In [ ]:
mouse_path = f'data_raw/jm/{mouse}/'

# now find all subfolders in mouse_path that start with '_a' in the end
subfolders = [f.path for f in os.scandir(mouse_path) if f.is_dir() and f.name.endswith('_a')]
subfolders.sort()
print(subfolders)


In [ ]:
# now in each in each of these find the fov tseries and append them to a list

wl = '830nm'

fov_tseries_830 = []

for p in subfolders:
    print(p)
    wl_path = os.path.join(p, 'fov', wl)
    # now in this path find a folder starting with 'TSeries' (there should only be one)
    tseries_folders = [f.path for f in os.scandir(wl_path) if f.is_dir() and f.name.startswith('TSeries')]
    tseries_folder = tseries_folders[0] 
    # now in this path find a file ending wiith '.tif'

    tif_files = [f.path for f in os.scandir(tseries_folder) if f.is_file() and f.name.endswith('.tif')]
    tif_file = tif_files[0]

    print(tif_file)

    # now read the tiff file, compute the mean across frames and append to fov_tseries
    tiff_data = tiff.imread(tif_file)
    tiff_mean = tiff_data.mean(axis=0)
    fov_tseries_830.append(tiff_mean)
    


In [ ]:
# now plot them
import matplotlib.pyplot as plt

ages = ['P8', 'P9', 'P10', 'P11', 'P12', 'P13']

fig, axs = plt.subplots(1, len(fov_tseries_830), figsize=(30, 5), dpi=500)
for i, ax in enumerate(axs):
    ax.imshow(fov_tseries_830[i], cmap='gray')
    ax.set_title(f'{ages[i]}')
    ax.axis('off')

# set suptitle
fig.suptitle(f'Mouse {mouse} FOVs at 830nm', fontsize=16)


In [ ]:
# now repeat the same for 1100nm
wl = '1100nm'

fov_tseries_1100 = []

for p in subfolders:
    print(p)
    wl_path = os.path.join(p, 'fov', wl)
    # now in this path find a folder starting with 'TSeries' (there should only be one)
    tseries_folders = [f.path for f in os.scandir(wl_path) if f.is_dir() and f.name.startswith('TSeries')]
    tseries_folder = tseries_folders[0] 
    # now in this path find a file ending wiith '.tif'

    tif_files = [f.path for f in os.scandir(tseries_folder) if f.is_file() and f.name.endswith('.tif')]
    tif_file = tif_files[0]

    print(tif_file)

    # now read the tiff file, compute the mean across frames and append to fov_tseries
    tiff_data = tiff.imread(tif_file)
    tiff_mean = tiff_data.mean(axis=0)
    fov_tseries_1100.append(tiff_mean)
    

In [ ]:
# now plot them

ages = ['P8', 'P9', 'P10', 'P11', 'P12', 'P13']

fig, axs = plt.subplots(1, len(fov_tseries_1100), figsize=(30, 5), dpi=500)
for i, ax in enumerate(axs):
    ax.imshow(fov_tseries_1100[i], cmap='gray')
    ax.set_title(f'{ages[i]}')
    ax.axis('off')

# set suptitle
fig.suptitle(f'Mouse {mouse} FOVs at 1100nm', fontsize=16)

In [ ]:

sat_percentile_red = 99.9
sat_percentile_green = 99.99

# now do a composite RGB image with 830nm as green and 1100nm as red
fig, axs = plt.subplots(1, len(fov_tseries_1100), figsize=(30, 5), dpi=500)
for i, ax in enumerate(axs):
    # create an RGB image
    rgb_image = np.zeros((*fov_tseries_1100[i].shape, 3), dtype=np.float32)
    # normalize and assign 1100nm to red channel
    red_max = np.percentile(fov_tseries_1100[i], sat_percentile_red)
    rgb_image[..., 0] = np.clip(fov_tseries_1100[i] / red_max, 0, 1)
    # normalize and assign 830nm to green channel
    green_max = np.percentile(fov_tseries_830[i], sat_percentile_green)
    rgb_image[..., 1] = np.clip(fov_tseries_830[i] / green_max, 0, 1)
    # blue channel remains zero
    ax.imshow(rgb_image)
    ax.set_title(f'{ages[i]}')
    ax.axis('off')
# set suptitle
fig.suptitle(f'Mouse {mouse} FOVs RGB composite (1100nm red, 830nm green)', fontsize=16)